In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/dm2021-lab2-hw2/tweets_DM.json
/kaggle/input/dm2021-lab2-hw2/sampleSubmission.csv
/kaggle/input/dm2021-lab2-hw2/data_identification.csv
/kaggle/input/dm2021-lab2-hw2/emotion.csv


In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
%matplotlib inline
sns.set(color_codes=True)

In [3]:
import pandas as pd
# import helpers.data_mining_helpers as dmh
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
import math
import nltk
import json
import re
import gensim
import string


from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.preprocessing import binarize
from sklearn import preprocessing, metrics, decomposition, pipeline, dummy
from sklearn.datasets import fetch_20newsgroups
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import LabelEncoder
from tensorflow import keras 

In [4]:
data_identification = pd.read_csv("../input/dm2021-lab2-hw2/data_identification.csv")
emotion = pd.read_csv('../input/dm2021-lab2-hw2/emotion.csv')
sampleSubmission = pd.read_csv("../input/dm2021-lab2-hw2/sampleSubmission.csv")

tweets = []
for line in open('../input/dm2021-lab2-hw2/tweets_DM.json', 'r'):
    tweets.append(json.loads(line))
    
dicts={}
for i in range(len(tweets)):
    dicts[tweets[i]['_source']['tweet']['tweet_id']]=tweets[i]['_source']['tweet']['text']
    
sent_train=[]
for i in range(len(emotion.tweet_id)):
    sent_train.append(dicts[emotion.tweet_id[i]])
    
emotion['sentence']=sent_train
training=emotion

d = {'id': training.tweet_id, 'sentence': training.sentence, 'emotion': training.emotion}
train = pd.DataFrame(data=d)

test_id=sampleSubmission['id']

sent_test=[]
for i in range(len(sampleSubmission.id)):
    sent_test.append(dicts[sampleSubmission.id[i]])
    
sampleSubmission['sentence']=sent_test
test=sampleSubmission

d = {'id': test.id, 'sentence': test.sentence}
test = pd.DataFrame(data=d)

train.to_csv('train_data.csv', index=False) 
test.to_csv('test_data.csv', index=False) 

In [5]:
train_balanced = train

In [6]:
train_data=train_balanced

test_data=test

y_train1=train_data.emotion.values

df1 = train_data.groupby(['emotion']).count()['sentence']

index=df1.index

y_train_conv=[]

for i in y_train1:
    for j in range(8):
        if i==index[j]:
            y_train_conv.append(j)
                        
y_train=np.array(y_train_conv)

In [7]:
text=train_data.sentence.values
text_tests=test_data.sentence.values

In [8]:
labels=y_train
labels

array([5, 2, 1, ..., 3, 4, 1])

In [9]:
from transformers import BertTokenizer,BertForSequenceClassification,AdamW

In [10]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased',do_lower_case = True)

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [11]:
input_ids = []
attention_mask = []
for i in text:
    encoded_data = tokenizer.encode_plus(
    i,
    add_special_tokens=True,
    max_length=64,
    pad_to_max_length = True,
    return_attention_mask= True,
    return_tensors='pt')
    input_ids.append(encoded_data['input_ids'])
    attention_mask.append(encoded_data['attention_mask'])
input_ids = torch.cat(input_ids,dim=0)
attention_mask = torch.cat(attention_mask,dim=0)
labels = torch.tensor(labels)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/opt/conda/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:2218: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [12]:
input_ids_tests = []
attention_mask_tests = []
for i in text_tests:
    encoded_data = tokenizer.encode_plus(
    i,
    add_special_tokens=True,
    max_length=64,
    pad_to_max_length = True,
    return_attention_mask= True,
    return_tensors='pt')
    input_ids_tests.append(encoded_data['input_ids'])
    attention_mask_tests.append(encoded_data['attention_mask'])
input_ids_tests = torch.cat(input_ids_tests,dim=0)
attention_mask_tests = torch.cat(attention_mask_tests,dim=0)

In [13]:
from torch.utils.data import DataLoader,SequentialSampler,RandomSampler,TensorDataset,random_split

In [14]:
dataset = TensorDataset(input_ids,attention_mask,labels)
train_size = int(0.8*len(dataset))
val_size = len(dataset) - train_size

train_dataset,val_dataset = random_split(dataset,[train_size,val_size])

print('Training Size - ',train_size)
print('Validation Size - ',val_size)

Training Size -  1164450
Validation Size -  291113


In [15]:
dataset_test = TensorDataset(input_ids_tests,attention_mask_tests)
print(len(dataset_test))
train_test = DataLoader(dataset_test, shuffle=False,batch_size = 16)

411972


In [16]:
train_dl = DataLoader(train_dataset,sampler = RandomSampler(train_dataset),
                     batch_size = 32)
val_dl = DataLoader(val_dataset,sampler = SequentialSampler(val_dataset),
                     batch_size = 32)

In [17]:
model = BertForSequenceClassification.from_pretrained(
'bert-base-uncased',
num_labels = 8,
output_attentions = False,
output_hidden_states = False)

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [18]:
import random

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [19]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(device)

cuda


In [20]:
optimizer = AdamW(model.parameters(),lr = 2e-5,eps=1e-8)

In [21]:
from transformers import get_linear_schedule_with_warmup
epochs = 1
total_steps = len(train_dl)*epochs
scheduler = get_linear_schedule_with_warmup(optimizer,num_warmup_steps=0,
                                           num_training_steps=total_steps)

In [22]:
def accuracy(preds,labels):
    pred_flat = np.argmax(preds,axis=1).flatten()
    label_flat = labels.flatten()
    return np.sum(pred_flat==label_flat)/len(label_flat)

In [23]:
def evaluate(dataloader_test):
    model.eval()
    loss_val_total = 0
    predictions,true_vals = [],[]
    for batch in dataloader_test:
        batch = tuple(b.to(device) for b in batch)
        inputs = {
            'input_ids':batch[0],
            'attention_mask': batch[1],
            'labels': batch[2]
        }
        with torch.no_grad():
            outputs = model(**inputs)
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()
        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    loss_val_avg = loss_val_total / len(dataloader_test)
    predictions = np.concatenate(predictions,axis=0)
    true_vals = np.concatenate(true_vals,axis=0)
    return loss_val_avg,predictions,true_vals

In [24]:
from tqdm.notebook import tqdm
torch.cuda.empty_cache()
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(train_dl, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(train_dl)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(val_dl)
    val_acc = accuracy(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'Accuracy: {val_acc}')


  0%|          | 0/1 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/36390 [00:00<?, ?it/s]


Epoch 1
Training loss: 1.030211106769886
Validation loss: 0.9412350455570494
Accuracy: 0.6602178535482786


In [25]:
output_dir = './'
model_to_save = model.module if hasattr(model, 'module') else model
model_to_save.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

('./tokenizer_config.json',
 './special_tokens_map.json',
 './vocab.txt',
 './added_tokens.json')

In [26]:
from transformers import BertTokenizer,BertForSequenceClassification
import torch
# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
output_dir = './'
tokenizer = BertTokenizer.from_pretrained(output_dir)
model_loaded = BertForSequenceClassification.from_pretrained(output_dir)

Loading BERT tokenizer...


In [27]:
def Sentiment(sent):
    output_dir = './'
    tokenizer = BertTokenizer.from_pretrained(output_dir)
    model_loaded = BertForSequenceClassification.from_pretrained(output_dir)
    encoded_dict = tokenizer.encode_plus(
                        sent, 
                        add_special_tokens = True,
                        max_length = 64,
                        pad_to_max_length = True,
                        return_attention_mask = True,
                        return_tensors = 'pt',
                   )
        
    input_id = encoded_dict['input_ids']

    attention_mask = encoded_dict['attention_mask']
    input_id = torch.LongTensor(input_id)
    attention_mask = torch.LongTensor(attention_mask)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model_loaded = model_loaded.to(device)
    input_id = input_id.to(device)
    attention_mask = attention_mask.to(device)

    with torch.no_grad():
        outputs = model_loaded(input_id, token_type_ids=None, attention_mask=attention_mask)

    logits = outputs[0]
    index = logits.argmax()
    return index

In [28]:
Sentiment('I am happy.')

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


tensor(4, device='cuda:0')

In [29]:
model.eval()
logit = []
summs=0
lens=len(dataset_test)/16
for batch in train_test:
    batchs = tuple(b.to(device) for b in batch)

    inputs = {'input_ids':      batchs[0],
              'attention_mask': batchs[1]
             }

    outputs = model(**inputs)
#     print(outputs)
    logit += torch.argmax(outputs[0], dim=1).cpu()
#     out.append(logit.item())
    if summs%1000==0:
        print(summs/lens)
    summs+=1

0.0
0.03883759090423621
0.07767518180847242
0.11651277271270863
0.15535036361694485
0.19418795452118104
0.23302554542541726
0.2718631363296535
0.3107007272338897
0.3495383181381259
0.3883759090423621
0.4272134999465983
0.4660510908508345
0.5048886817550707
0.543726272659307
0.5825638635635432
0.6214014544677794
0.6602390453720156
0.6990766362762518
0.737914227180488
0.7767518180847242
0.8155894089889604
0.8544269998931966
0.8932645907974328
0.932102181701669
0.9709397726059052


In [30]:
logit

[tensor(4),
 tensor(4),
 tensor(5),
 tensor(7),
 tensor(4),
 tensor(2),
 tensor(4),
 tensor(1),
 tensor(4),
 tensor(4),
 tensor(5),
 tensor(4),
 tensor(1),
 tensor(4),
 tensor(5),
 tensor(7),
 tensor(4),
 tensor(4),
 tensor(1),
 tensor(6),
 tensor(4),
 tensor(1),
 tensor(1),
 tensor(4),
 tensor(1),
 tensor(5),
 tensor(4),
 tensor(4),
 tensor(5),
 tensor(0),
 tensor(4),
 tensor(1),
 tensor(5),
 tensor(5),
 tensor(4),
 tensor(4),
 tensor(7),
 tensor(2),
 tensor(4),
 tensor(2),
 tensor(4),
 tensor(3),
 tensor(2),
 tensor(4),
 tensor(2),
 tensor(5),
 tensor(4),
 tensor(1),
 tensor(3),
 tensor(4),
 tensor(5),
 tensor(5),
 tensor(4),
 tensor(4),
 tensor(1),
 tensor(2),
 tensor(5),
 tensor(4),
 tensor(4),
 tensor(1),
 tensor(5),
 tensor(5),
 tensor(4),
 tensor(5),
 tensor(5),
 tensor(5),
 tensor(1),
 tensor(1),
 tensor(7),
 tensor(5),
 tensor(4),
 tensor(5),
 tensor(4),
 tensor(4),
 tensor(5),
 tensor(4),
 tensor(2),
 tensor(4),
 tensor(4),
 tensor(4),
 tensor(4),
 tensor(2),
 tensor(5),
 ten

In [31]:
pred=[]
for i in logit:
    pred.append(df1.index[i])

In [32]:
test_data['emotion']=pred

In [33]:
sb=test_data.loc[:,['id','emotion']]
sb

,id,emotion
0,0x2c7743,joy
1,0x2c1eed,joy
2,0x2826ea,sadness
3,0x356d9a,trust
4,0x20fd95,joy
...,...,...
411967,0x351857,disgust
411968,0x2c028e,anger
411969,0x1f2430,disgust
411970,0x2be24e,joy


In [34]:
sb.to_csv('submit.csv', index=False) 